Lambda School Data Science, Unit 2: Predictive Modeling

# Kaggle Challenge, Module 3


## Assignment
- [ ] [Review requirements for your portfolio project](https://lambdaschool.github.io/ds/unit2), then submit your dataset.
- [ ] Continue to participate in our Kaggle challenge. 
- [ ] Use scikit-learn for hyperparameter optimization with RandomizedSearchCV.
- [ ] Submit your predictions to our Kaggle competition. (Go to our Kaggle InClass competition webpage. Use the blue **Submit Predictions** button to upload your CSV file. Or you can use the Kaggle API to submit your predictions.)
- [ ] Commit your notebook to your fork of the GitHub repo.

## Stretch Goals

### Reading
- Jake VanderPlas, [Python Data Science Handbook, Chapter 5.3](https://jakevdp.github.io/PythonDataScienceHandbook/05.03-hyperparameters-and-model-validation.html), Hyperparameters and Model Validation
- Jake VanderPlas, [Statistics for Hackers](https://speakerdeck.com/jakevdp/statistics-for-hackers?slide=107)
- Ron Zacharski, [A Programmer's Guide to Data Mining, Chapter 5](http://guidetodatamining.com/chapter5/), 10-fold cross validation
- Sebastian Raschka, [A Basic Pipeline and Grid Search Setup](https://github.com/rasbt/python-machine-learning-book/blob/master/code/bonus/svm_iris_pipeline_and_gridsearch.ipynb)
- Peter Worcester, [A Comparison of Grid Search and Randomized Search Using Scikit Learn](https://blog.usejournal.com/a-comparison-of-grid-search-and-randomized-search-using-scikit-learn-29823179bc85)

### Doing
- Add your own stretch goals!
- Try other [categorical encodings](https://contrib.scikit-learn.org/categorical-encoding/). See the previous assignment notebook for details.
- In additon to `RandomizedSearchCV`, scikit-learn has [`GridSearchCV`](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html). Another library called scikit-optimize has [`BayesSearchCV`](https://scikit-optimize.github.io/notebooks/sklearn-gridsearchcv-replacement.html). Experiment with these alternatives.
- _[Introduction to Machine Learning with Python](http://shop.oreilly.com/product/0636920030515.do)_ discusses options for "Grid-Searching Which Model To Use" in Chapter 6:

> You can even go further in combining GridSearchCV and Pipeline: it is also possible to search over the actual steps being performed in the pipeline (say whether to use StandardScaler or MinMaxScaler). This leads to an even bigger search space and should be considered carefully. Trying all possible solutions is usually not a viable machine learning strategy. However, here is an example comparing a RandomForestClassifier and an SVC ...

The example is shown in [the accompanying notebook](https://github.com/amueller/introduction_to_ml_with_python/blob/master/06-algorithm-chains-and-pipelines.ipynb), code cells 35-37. Could you apply this concept to your own pipelines?


### BONUS: Stacking!

Here's some code you can use to "stack" multiple submissions, which is another form of ensembling:

```python
import pandas as pd

# Filenames of your submissions you want to ensemble
files = ['submission-01.csv', 'submission-02.csv', 'submission-03.csv']

target = 'status_group'
submissions = (pd.read_csv(file)[[target]] for file in files)
ensemble = pd.concat(submissions, axis='columns')
majority_vote = ensemble.mode(axis='columns')[0]

sample_submission = pd.read_csv('sample_submission.csv')
submission = sample_submission.copy()
submission[target] = majority_vote
submission.to_csv('my-ultimate-ensemble-submission.csv', index=False)
```

In [7]:
# Ignore this Numpy warning when using Plotly Express:
# FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
import warnings
warnings.filterwarnings(action='ignore', category=FutureWarning, module='numpy')

In [13]:
%%capture
import sys

# If you're on Colab:
if 'google.colab' in sys.modules:
    DATA_PATH = 'https://raw.githubusercontent.com/LambdaSchool/DS-Unit-2-Kaggle-Challenge/master/data/'
    !pip install category_encoders==2.*

# If you're working locally:
else:
    DATA_PATH = '../data/'

In [178]:
# Merge train_features.csv & train_labels.csv
train = pd.merge(pd.read_csv(DATA_PATH+'waterpumps/train_features.csv'), 
                 pd.read_csv(DATA_PATH+'waterpumps/train_labels.csv'))

# Read test_features.csv & sample_submission.csv
test = pd.read_csv(DATA_PATH+'waterpumps/test_features.csv')
sample_submission = pd.read_csv(DATA_PATH+'waterpumps/sample_submission.csv')

In [179]:
train = pd.merge(pd.read_csv(DATA_PATH+'waterpumps/train_features.csv'), 
                 pd.read_csv(DATA_PATH+'waterpumps/train_labels.csv'))
test = pd.read_csv(DATA_PATH+'waterpumps/test_features.csv')
sample_submission = pd.read_csv(DATA_PATH+'waterpumps/sample_submission.csv')
final = train.copy()

train.shape, test.shape, final.shape

((59400, 41), (14358, 40), (59400, 41))

In [180]:
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder, LabelEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV
import pandas as pd
from sklearn.model_selection import train_test_split
import numpy as np
import category_encoders as ce
import matplotlib.pyplot as plt
from sklearn.tree import DecisionTreeClassifier
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
import pandas_profiling
from sklearn.feature_selection import f_regression, SelectKBest
from sklearn.linear_model import Ridge
from sklearn.model_selection import cross_val_score
from scipy.stats import randint, uniform
from sklearn.pipeline import make_pipeline

In [181]:
from sklearn.model_selection import train_test_split
train, val = train_test_split(train, train_size=0.80, test_size=0.20, 
                              stratify=train['status_group'], random_state=7)
train.shape, val.shape

((47520, 41), (11880, 41))

In [182]:
def wrangle(X):
    """Wrangle train, validate, and test sets in the same way"""
    
    # Prevent SettingWithCopyWarning
    X = X.copy()
    
    # About 3% of the time, latitude has small values near zero,
    # outside Tanzania, so we'll treat these values like zero.
    X['latitude'] = X['latitude'].replace(-2e-08, 0)
    X['gps_height'] = X['gps_height'].replace([-90], X['gps_height'].mean())
    # When columns have zeros and shouldn't, they are like null values.
    # So we will replace the zeros with nulls, and impute missing values later.
    cols_with_zeros = ['longitude', 'latitude', 'construction_year', 'population']
    for col in cols_with_zeros:
        X[col] = X[col].replace(0, np.nan)
    #X.date_recorded = X['date_recorded'].str[:4]#.astype('float64')
    #X.construction_year = X['construction_year'].astype('str').str[:4].astype('float64')
    # quantity & quantity_group are duplicates, num_private has no discernible use.
    X = X.drop(columns=['quantity_group','num_private', 'recorded_by', 'payment_type', 'id'])
    X['date_recorded'] = pd.to_datetime(X['date_recorded'], infer_datetime_format=True)
    
    # Extract components from date_recorded, then drop the original column
    X['year_recorded'] = X['date_recorded'].dt.year
    X['month_recorded'] = X['date_recorded'].dt.month
    X['day_recorded'] = X['date_recorded'].dt.day
    X = X.drop(columns='date_recorded')
    X['years'] = X['year_recorded'] - X['construction_year']
    X['years_MISSING'] = X['years'].isnull()
    # return the wrangled dataframe
    return X

final = wrangle(final)
train = wrangle(train)
val = wrangle(val)
test = wrangle(test)

In [183]:
final.shape[0]

59400

In [184]:
# The status_group column is the target
target = 'status_group'

# Get a dataframe with all train columns except the target & id
train_features = train.drop(columns=[target])

# Get a series with the cardinality of the nonnumeric features
cardinality = train_features.select_dtypes(exclude='number').nunique()

# Get a list of the numeric features
numeric_features = train_features.select_dtypes(include='number').columns.tolist()
numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('scaler', StandardScaler())])

# Get a list of all categorical features with cardinality <= 50

categorical_features = cardinality[cardinality <= 130].index.tolist()
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='constant', fill_value='missing')),
    ('onehot', ce.OneHotEncoder(use_cat_names=True)),
    ('kbest', SelectKBest(k = 100))])

ordinal_features = cardinality[cardinality >= 130].index.tolist()
ordinal_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='constant', fill_value='missing')),
    ('ordinal', ce.OrdinalEncoder()),
    ('kbest', SelectKBest(k = 'all'))])

target_features = target
target_transformer = Pipeline(steps=[('label', LabelEncoder())])

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features),
        ('ord', ordinal_transformer, ordinal_features)])

# ('tar', target_transformer, target_features)

features = numeric_features + categorical_features + ordinal_features

X_train = train[features]
y_train = train[target]
X_val = val[features]
y_val = val[target]
X_test = test[features]
X_final = final[features]
y_final = final[target]


#X_final = X_final[(X_final[numeric_features] >= np.percentile(X_final[numeric_features], 0.5))]
# X_train = train.drop(columns=target)
# y_train = train.drop(columns=features)
# X_val = val.drop(columns=target)
# y_val = val[target]
# X_test = test
# X_final = final.drop(columns=target)
# y_final = final[target]

# Combine the lists 
# features = numeric_features + categorical_features + ordinal_features
print(features)

['amount_tsh', 'gps_height', 'longitude', 'latitude', 'region_code', 'district_code', 'population', 'construction_year', 'year_recorded', 'month_recorded', 'day_recorded', 'years', 'basin', 'region', 'lga', 'public_meeting', 'scheme_management', 'permit', 'extraction_type', 'extraction_type_group', 'extraction_type_class', 'management', 'management_group', 'payment', 'water_quality', 'quality_group', 'quantity', 'source', 'source_type', 'source_class', 'waterpoint_type', 'waterpoint_type_group', 'years_MISSING', 'funder', 'installer', 'wpt_name', 'subvillage', 'ward', 'scheme_name']


In [185]:
numeric_features

['amount_tsh',
 'gps_height',
 'longitude',
 'latitude',
 'region_code',
 'district_code',
 'population',
 'construction_year',
 'year_recorded',
 'month_recorded',
 'day_recorded',
 'years']

In [186]:
y_train.head()

26660        functional
46901        functional
1058         functional
9559     non functional
35556        functional
Name: status_group, dtype: object

In [87]:
X_train.head()

,amount_tsh,gps_height,longitude,latitude,region_code,district_code,population,construction_year,year_recorded,month_recorded,...,source_class,waterpoint_type,waterpoint_type_group,years_MISSING,funder,installer,wpt_name,subvillage,ward,scheme_name
26660,0.0,1138.0,36.901195,-3.391070,2,7,350.0,2009.0,2013,2,...,surface,communal standpipe,communal standpipe,False,Rotary Club,Rotary Club,Kitefu Primary School,Tanesco,Maji ya Chai,NaN
46901,3000.0,910.0,35.366883,-7.642936,11,1,1.0,1992.0,2011,3,...,groundwater,communal standpipe,communal standpipe,False,Danida,DANID,none,Ndorobo B,Mlowa,Mlowa
1058,1500.0,682.0,37.512633,-3.542461,3,2,250.0,2011.0,2013,2,...,groundwater,communal standpipe,communal standpipe,False,Government Of Tanzania,District council,Kwa Salim Idd,Kitopeni,Kileo,Kifaru water Supply
9559,0.0,370.0,38.688599,-4.847517,4,2,150.0,1975.0,2011,3,...,groundwater,communal standpipe,communal standpipe,False,Government Of Tanzania,RWE,Msikitini,Dukani,Kizara,Bombomajimoto water
35556,0.0,0.0,34.156026,-3.104921,17,6,NaN,NaN,2012,10,...,groundwater,hand pump,hand pump,True,Rwssp,WEDECO,Polewalandi,Makungu,Kisesa,None


In [187]:
# pipeline = Pipeline(steps=[('preprocessor', preprocessor),
#                       ('classifier', RandomForestRegressor(random_state=7))])

# pipeline = make_pipeline(
#     preprocessor, 
#     RandomForestRegressor(random_state=7)
# )

pipeline = Pipeline(steps=[('preprocessor', preprocessor),
                      ('classifier', LogisticRegression(solver='lbfgs'))])

pipeline.fit(X_train, y_train)

# Score on val
#print('Validation Accuracy', pipeline.score(X_val, y_val))

# Predict on test
#y_pred = pipeline.predict(X_test)

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:947: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:947: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:947: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)


Pipeline(memory=None,
         steps=[('preprocessor',
                 ColumnTransformer(n_jobs=None, remainder='drop',
                                   sparse_threshold=0.3,
                                   transformer_weights=None,
                                   transformers=[('num',
                                                  Pipeline(memory=None,
                                                           steps=[('imputer',
                                                                   SimpleImputer(add_indicator=False,
                                                                                 copy=True,
                                                                                 fill_value=None,
                                                                                 missing_values=nan,
                                                                                 strategy='most_frequent',
                                                      

In [188]:
#Score on val
print('Validation Accuracy', pipeline.score(X_val, y_val))

#Predict on test
y_pred = pipeline.predict(X_test)

Validation Accuracy 0.6749158249158249


In [189]:
pipeline = Pipeline(steps=[('preprocessor', preprocessor),
                      ('classifier', RandomForestClassifier(n_estimators=100, random_state=42, n_jobs=-1))])
pipeline.fit(X_train, y_train)

print('Train Accuracy', pipeline.score(X_train, y_train))
print('Validation Accuracy', pipeline.score(X_val, y_val))

Train Accuracy 0.9999789562289563
Validation Accuracy 0.8168350168350168


My baseline here is the same as the submission I got in kaggle. This accuracy came out to be 0.81501.

In [190]:
pipeline.fit(X_final, y_final)
print('Final Accuracy', pipeline.score(X_final, y_final))
y_pred = pipeline.predict(X_test)
sample_submission = pd.read_csv(DATA_PATH+'waterpumps/sample_submission.csv')
submission = sample_submission.copy()
submission['status_group'] = y_pred
submission.to_csv('your-submission-filename.csv', index=False)
!kaggle competitions submit -c ds8-predictive-modeling-challenge -f your-submission-filename.csv -m "Using preprocessing to encode everything in a different manner"

Final Accuracy 0.999983164983165
Successfully submitted to DS8 Predictive Modeling Challenge



  0%|          | 0.00/279k [00:00<?, ?B/s]
 40%|####      | 112k/279k [00:00<00:00, 1.14MB/s]
 78%|#######7  | 216k/279k [00:00<00:00, 1.08MB/s]
100%|##########| 279k/279k [00:01<00:00, 167kB/s] 


In [100]:
pipeline = Pipeline(steps=[('preprocessor', preprocessor),
                           ('target_encoder', ce.TargetEncoder),
                           ('classifier', RandomForestRegressor(random_state=7))])
pipeline.fit(X_train, y_train)

print('Train Accuracy', pipeline.score(X_train, y_train))
print('Validation Accuracy', pipeline.score(X_val, y_val))

TypeError: exceptions must derive from BaseException

In [193]:
# pipeline = make_pipeline(
#     ce.TargetEncoder(), 
#     SimpleImputer(), 
#     RandomForestRegressor(random_state=42)
# )
from scipy.stats import randint, uniform
pipeline = Pipeline(steps=[('preprocessor', preprocessor),
                          # ('targetencoder',ce.TargetEncoder()),
                      ('classifier', RandomForestClassifier(n_estimators=100, random_state=42,n_jobs=-1))])
param_distributions = {
#    'targetencoder__min_samples_leaf': randint(1, 1000), 
#    'targetencoder__smoothing': uniform(1, 1000), 
#    'simpleimputer__strategy': ['mean', 'median'], 
    #'preprocessor__num__imputer__strategy': ['mean', 'most_frequent'],
    'preprocessor__cat__kbest__k': range(0, len(X_train.columns)+1),
    'preprocessor__ord__kbest__k': range(5,7),
    'classifier__n_estimators': randint(200, 300), 
    'classifier__max_depth': [60, 65, 70], 
    'classifier__max_features': uniform(0, .5), 
}

# If you're on Colab, decrease n_iter & cv parameters
search = RandomizedSearchCV(
    pipeline, 
    param_distributions=param_distributions, 
    n_iter=5, 
    cv=3, 
    #scoring='neg_mean_absolute_error', 
    verbose=10, 
    return_train_score=True, 
    n_jobs=-1
)

search.fit(X_final, y_final);

print('Best hyperparameters', search.best_params_)
print('Cross-validation MAE', -search.best_score_)

Fitting 3 folds for each of 5 candidates, totalling 15 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 out of  15 | elapsed:   51.5s remaining:  5.6min
[Parallel(n_jobs=-1)]: Done   4 out of  15 | elapsed:   56.6s remaining:  2.6min
[Parallel(n_jobs=-1)]: Done   6 out of  15 | elapsed:   59.8s remaining:  1.5min
[Parallel(n_jobs=-1)]: Done   8 out of  15 | elapsed:  1.1min remaining:   55.6s
[Parallel(n_jobs=-1)]: Done  10 out of  15 | elapsed:  1.7min remaining:   51.1s
[Parallel(n_jobs=-1)]: Done  12 out of  15 | elapsed:  1.9min remaining:   28.5s
[Parallel(n_jobs=-1)]: Done  15 out of  15 | elapsed:  2.1min finished


Best hyperparameters {'classifier__max_depth': 70, 'classifier__max_features': 0.23786198843999357, 'classifier__n_estimators': 230, 'preprocessor__cat__kbest__k': 39, 'preprocessor__ord__kbest__k': 6}
Cross-validation MAE -0.8037205387205387


In [149]:
pipeline = search.best_estimator_
y_pred = pipeline.predict(X_test)
sample_submission = pd.read_csv(DATA_PATH+'waterpumps/sample_submission.csv')
submission = sample_submission.copy()
submission['status_group'] = y_pred
submission.to_csv('your-submission-filename.csv', index=False)
!kaggle competitions submit -c ds8-predictive-modeling-challenge -f your-submission-filename.csv -m "Using randomized searchCV got a .79 MAE"

Successfully submitted to DS8 Predictive Modeling Challenge



  0%|          | 0.00/278k [00:00<?, ?B/s]
 43%|####3     | 120k/278k [00:00<00:00, 1.17MB/s]
 78%|#######7  | 216k/278k [00:00<00:00, 1.10MB/s]
100%|##########| 278k/278k [00:01<00:00, 146kB/s] 


In [ ]:
# param_grid = {
#     'preprocessor__num__imputer__strategy': ['mean', 'median'],
#     'classifier__C': [0.1, 1.0, 10, 100],
# }

# grid_search = GridSearchCV(clf, param_grid, cv=10, iid=False)
# grid_search.fit(X_train, y_train)

# print(("best logistic regression from grid search: %.3f"
#        % grid_search.score(X_test, y_test)))